# 구글 플레이 스토어 리뷰 웹 크롤러
- Contributor: Tony Park
- References
    - Tistory Blog | https://heytech.tistory.com/293
    - Github | https://github.com/park-gb/playstore-review-crawler
- Last Updated@2022-07-19

# 초기 설정

## 크롬 드라이버 세팅

In [1]:
# chrome_driver = '../chromedriver.exe' # 파일 확장자 이름 표기
chrome_driver = '../chromedriver' # 파일 확장자 이름 미표기

## 수집 앱 URL

In [2]:
URL = 'https://play.google.com/store/apps/details?id=com.github.android'

# 패키지 import

In [3]:
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from time import sleep
import random
import pandas as pd

# 무한 스크롤 함수

In [4]:
def scroll(modal):
    try:        
        # 스크롤 높이 받아오기
        last_height = driver.execute_script("return arguments[0].scrollHeight", modal)
        while True:
            pause_time = random.uniform(0.5, 0.8)
            # 최하단까지 스크롤
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", modal)
            # 페이지 로딩 대기
            time.sleep(pause_time)
            # 무한 스크롤 동작을 위해 살짝 위로 스크롤
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight-50);", modal)
            time.sleep(pause_time)
            # 스크롤 높이 새롭게 받아오기
            new_height = driver.execute_script("return arguments[0].scrollHeight", modal)
            try:
                # '더보기' 버튼 있을 경우 클릭
                all_review_button = driver.find_element_by_xpath('/html/body/div[1]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div[2]/div/span/span').click()
            except:
                # 스크롤 완료 경우
                if new_height == last_height:
                    print("스크롤 완료")
                    break
                last_height = new_height
                
    except Exception as e:
        print("에러 발생: ", e)

# 데이터 크롤링

In [5]:
# 크롬 드라이버 세팅
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
driver = webdriver.Chrome(chrome_driver, options = chrome_options)
# 페이지 열기
driver.get(URL)
# 페이지 로딩 대기
wait = WebDriverWait(driver, 5)

# '리뷰 모두 보기' 버튼 렌더링 확인(path 수정 @2022-06-22)
all_review_button_xpath = '/html/body/c-wiz[2]/div/div/div[1]/div[2]/div/div[1]/c-wiz[4]/section/div/div/div[5]/div/div/button/span'
button_loading_wait = wait.until(EC.element_to_be_clickable((By.XPATH, all_review_button_xpath)))
# '리뷰 모두 보기' 버튼 클릭
driver.find_element_by_xpath(all_review_button_xpath).click()

# '리뷰 모두 보기' 페이지 렌더링 대기
all_review_page_xpath = '/html/body/div[4]/div[2]/div/div/div/div/div[2]'
page_loading_wait = wait.until(EC.element_to_be_clickable((By.XPATH, all_review_page_xpath)))

# 페이지 무한 스크롤 다운
modal = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='fysCi']")))
scroll(modal)

/var/folders/w1/1zs4c_t97jx0d485bswtnm3m0000gn/T/ipykernel_2215/3029485729.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver, options = chrome_options)
/var/folders/w1/1zs4c_t97jx0d485bswtnm3m0000gn/T/ipykernel_2215/3029485729.py:16: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(all_review_button_xpath).click()
/var/folders/w1/1zs4c_t97jx0d485bswtnm3m0000gn/T/ipykernel_2215/3953322799.py:18: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  all_review_button = driver.find_element_by_xpath('/html/body/div[1]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div[2]/div/span/span').click()


스크롤 완료


# HTML Parsing

In [6]:
# html parsing하기
html_source = driver.page_source
soup_source = BeautifulSoup(html_source, 'html.parser')

# 크롬 드라이버 종료

In [7]:
driver.quit()

# HTML 데이터 저장

In [8]:
# html 데이터 저장
with open("../dataset/data_html.html", "w", encoding = 'utf-8') as file:
    file.write(str(soup_source))

## 데이터프레임 변환

In [9]:
# 리뷰 데이터 클래스 접근
review_source = soup_source.find_all(class_ = 'RHo1pe')
# 리뷰 데이터 저장용 배열
dataset = []
# 데이터 넘버링을 위한 변수
review_num = 0 
# 리뷰 1개씩 접근해 정보 추출
for review in review_source:
    review_num+=1
    # 리뷰 등록일 데이터 추출
    date_full = review.find_all(class_ = 'bp9Aid')[0].text
    date_year = date_full[0:4] # 연도 데이터 추출
    # 해당 단어가 등장한 인덱스 추출
    year_index = date_full.find('년')
    month_index = date_full.find('월')
    day_index = date_full.find('일')
    
    date_month = str(int(date_full[year_index+1:month_index])) # 월(Month) 데이터 추출
    # 월 정보가 1자리의 경우 앞에 0 붙이기(e.g., 1월 -> 01월)
    if len(date_month) == 1:
        date_month = '0' + date_month
    
    date_day = str(int(date_full[month_index+1:day_index])) # 일(Day) 데이터 추출 
    # 일 정보가 1자리의 경우 앞에 0 붙여줌(e.g., 7일 -> 07일)
    if len(date_day) == 1:
        date_day = '0' + date_day
    
    # 리뷰 등록일 full version은 최종적으로 yyyymmdd 형태로 저장
    date_full = date_year + date_month + date_day
    user_name = review.find_all(class_ = 'X5PpBb')[0].text # 닉네임 데이터 추출
    rating = review.find_all(class_ = "iXRFPc")[0]['aria-label'][10] # 평점 데이터 추출
    content = review.find_all(class_ = 'h3YV2d')[0].text # 리뷰 데이터 추출

    data = {
        "id": review_num, 
        "date": date_full,
        "dateYear": date_year,
        "dateMonth": date_month,
        "dateDay": date_day,
        "rating": rating,
        "userName": user_name,
        "content": content
    }
    dataset.append(data)

## 데이터프레임 저장

In [10]:
df = pd.DataFrame(dataset)
df.to_csv('../dataset/review_dataset.csv', encoding = 'utf-8-sig') # csv 파일로 저장

# 리뷰 데이터 불러오기

In [11]:
# 저장한 리뷰 정보 불러오기
df = pd.read_csv('../dataset/review_dataset.csv', encoding = 'utf-8-sig')
df = df.drop(['Unnamed: 0'], axis = 1) # 불필요한 칼럼 삭제
df

,id,date,dateYear,dateMonth,dateDay,rating,userName,content
0,1,20220630,2022,6,30,5,JH son,모바일 웹 페이지보자 직관적임 ++한국어 지원되면 좋을 듯. 그래도 나름 공식 앱인데ㅜ
1,2,20220228,2022,2,28,5,김기철,"깃허브. 코딩으로 뭘 개발하는 팀, 인가? 코딩은 배우기 어렵고 사용하기도 힘들고...."
2,3,20220807,2022,8,7,5,전현배,매우 편리함
3,4,20200613,2020,6,13,5,참티즈는말지않긔,간단히 멘션 확인같은거 할때 좋음. 큰 코드는 당연하게도 모바일로는 힘들다.
4,5,20210926,2021,9,26,5,tica,이것저것 할게 많은 GitHub의 특성상 모바일 웹 페이지로 접속하는 게 더 편하긴...
...,...,...,...,...,...,...,...,...
69,70,20200318,2020,3,18,5,Google 사용자,Good
70,71,20220129,2022,1,29,4,Charles Johnson,Pretty useful apartment having navigate back m...
71,72,20200318,2020,3,18,5,Google 사용자,꺄악~~
72,73,20200522,2020,5,22,5,Bear Black,Good!!
